<a href="https://colab.research.google.com/github/mekabuko/DS_TIL/blob/main/Aidemy_Option/21_%E6%B7%B1%E5%B1%A4%E5%BC%B7%E5%8C%96%E5%AD%A6%E7%BF%92%E7%99%BA%E5%B1%95/21_%E6%B7%B1%E5%B1%A4%E5%BC%B7%E5%8C%96%E5%AD%A6%E7%BF%92%E7%99%BA%E5%B1%95_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install tf-agents==0.6.0

     |████████████████████████████████| 1.1MB 4.0MB/s 


In [3]:
import numpy as np
from tensorflow.compat.v1.train import AdamOptimizer
from tf_agents.agents.dqn import dqn_agent
from tf_agents.environments import suite_gym, tf_py_environment
from tf_agents.networks import q_network
from tf_agents.policies import random_tf_policy
from tf_agents.replay_buffers import tf_uniform_replay_buffer
from tf_agents.trajectories import trajectory
from tf_agents.utils import common

# 訓練用の環境と評価用の環境の作成
ENV_NAME = 'CartPole-v0'
env = suite_gym.load(ENV_NAME)
train_env = tf_py_environment.TFPyEnvironment(env)
eval_py_env = suite_gym.load(ENV_NAME)
eval_env = tf_py_environment.TFPyEnvironment(eval_py_env)

# 多層ニューラルネットワーク(Qネットワーク)の構築
q_net = q_network.QNetwork(
    train_env.observation_spec(),
    train_env.action_spec(),
    fc_layer_params=(16,16,16))


# DQNエージェントの作成
tf_agent = dqn_agent.DqnAgent(
    train_env.time_step_spec(),
    train_env.action_spec(),
    q_network=q_net,
    optimizer=AdamOptimizer(learning_rate=1e-3),
    td_errors_loss_fn=common.element_wise_squared_loss)
tf_agent.initialize()

# 履歴の構築
replay_buffer = tf_uniform_replay_buffer.TFUniformReplayBuffer(
   data_spec=tf_agent.collect_data_spec,
   batch_size=train_env.batch_size,
   max_length=100000)


# 評価値を計算する関数です
def compute_avg_return(environment, policy, num_episodes=10):
    total_return = 0.0
    for _ in range(num_episodes):
        time_step = environment.reset()
        episode_return = 0.0
        while not time_step.is_last():
            action_step = policy.action(time_step)
            time_step = environment.step(action_step.action)
            episode_return += time_step.reward
        total_return += episode_return
    avg_return = total_return / num_episodes
    return avg_return.numpy()[0]

# 経験を集めてリプレイバッファに蓄積する関数です
def collect_step(environment, policy):
    time_step = environment.current_time_step()
    action_step = policy.action(time_step)
    next_time_step = environment.step(action_step.action)
    traj = trajectory.from_transition(time_step, action_step, next_time_step)
    replay_buffer.add_batch(traj)

# 初めにランダムな方策を実行してリプレイバッファに蓄積します
initial_collect_steps=1000
random_policy = random_tf_policy.RandomTFPolicy(train_env.time_step_spec(), train_env.action_spec())
for _ in range(initial_collect_steps):
    collect_step(train_env, random_policy)
dataset = replay_buffer.as_dataset(
   num_parallel_calls=3, sample_batch_size=64, num_steps=2).prefetch(3)
iterator = iter(dataset)

num_eval_episodes = 5  
eval_interval = 1000 
num_iterations = 20000
# 訓練前にモデルを評価します
avg_return = compute_avg_return(eval_env, tf_agent.policy, num_eval_episodes)
returns = [avg_return]
for step in range(1, num_iterations+1):
    # エージェントが環境(CartPole-v0)と相互作用して得た経験をリプレイバッファに追加します
    collect_step(train_env, tf_agent.collect_policy)
    # リプレイバッファから経験を取り出して学習を行います
    experience, unused_info = next(iterator)
    train_loss = tf_agent.train(experience)
    # step数に応じてモデルの評価(テスト)を行います
    if step % eval_interval == 0:
        avg_return = compute_avg_return(eval_env, tf_agent.policy, num_eval_episodes)
        print('step = {0}: Average Return = {1}'.format(step, avg_return))
        returns.append(avg_return)

Instructions for updating:
Use `as_dataset(..., single_deterministic_pass=False) instead.


Instructions for updating:
Use `as_dataset(..., single_deterministic_pass=False) instead.


Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.foldr(fn, elems, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.foldr(fn, elems))


Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.foldr(fn, elems, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.foldr(fn, elems))


step = 1000: Average Return = 20.200000762939453
step = 2000: Average Return = 109.19999694824219
step = 3000: Average Return = 183.60000610351562
step = 4000: Average Return = 200.0
step = 5000: Average Return = 200.0
step = 6000: Average Return = 172.39999389648438
step = 7000: Average Return = 122.5999984741211
step = 8000: Average Return = 10.0
step = 9000: Average Return = 200.0
step = 10000: Average Return = 52.0
step = 11000: Average Return = 200.0
step = 12000: Average Return = 171.60000610351562
step = 13000: Average Return = 167.8000030517578
step = 14000: Average Return = 200.0
step = 15000: Average Return = 200.0
step = 16000: Average Return = 200.0
step = 17000: Average Return = 200.0
step = 18000: Average Return = 200.0
step = 19000: Average Return = 200.0
step = 20000: Average Return = 200.0
